In [3]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from data.data_accessors import tmprss2_to_pandas
import numpy as np
from rdkit import Chem, DataStructs
import matplotlib.pyplot as plt

from sklearn.svm import LinearSVC
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# collect dataset
dataset = tmprss2_to_pandas()

# visualize dataset
plt.figure()
hist, bins, _ = plt.hist(dataset.Activity, bins=8)
plt.clf()
logbins = np.logspace(np.log10(bins[0]), np.log10(bins[-1]), len(bins))
plt.figure()
plt.hist(dataset.Activity, bins=logbins)
plt.xscale('log')
plt.xlabel('Activity (log scale)')

# calculate fingerprints
dataset['fingerprints'] = dataset.SMILES.apply(lambda s: Chem.RDKFingerprint(Chem.MolFromSmiles(s)))
# convert fingerprints to arrays
X = np.zeros([len(dataset), 2048])
for i in range(len(dataset)):
    DataStructs.ConvertToNumpyArray(dataset.fingerprints.iloc[i], X[i])

X = np.array(dataset.fingerprints.data)
y = np.array(dataset.Activity)
y = dataset.Activity.values


#create binary dataset to train linear SVM to predict what structures will have activity values below 1 micromolar
y_binary = y

for i in range(y_binary.size()):
    if elem > 1000:
        y_binary[i] = 0
    else:
        y_binary[i] = 1
        
#split training and testing data for each dataset
train_X_binary, test_X_binary, train_y_binary, test_y_binary = train_test_split(X, y_binary, test_size=0.2)
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2)


#linear SVM
clf = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5))
clf.fit(train_X_binary, train_y_binary)

pred = clf.predict(test_X_binary)
print("Accuracy:{}".format(accuracy_score(test_y_binary, pred)))

#SVM regression
svmReg = make_pipeline(StandardScaler(), LinearSVR(random_state=0, tol=1e-5))
regr.fit(train_X, train_y)

pred = regr.predict(test_X)
RMSE = mean_squared_error(test_y, pred, squared=False)


ModuleNotFoundError: No module named 'data.data_accessors'